In [2]:
import json
import boto3
import pandas as pd

from utils.analysis_utils import classify_one_feedback
from utils.database import get_field_value
from pymongo import MongoClient
import certifi

In [3]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"Prod/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)
mongo_client = MongoClient(MONGO_CONNECTION_STRING,tlsCAFile=certifi.where())

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

# Generating Elementary Subjects

After creating all the extractions, our objective is to generate elementary subjects.

We use the following method:

1. For each extraction, we attempt to classify it among the existing elementary subjects.
2. If no subject matches, we generate a new one.
3. As we progress, we also verify that we are not generating duplicates.

## Process

We classify and generate our elementary subjects progressively:

1. Classify each extraction
2. Generate new subjects as needed
3. Check for duplicates

By the end of this process, after classifying everything, we will have generated all our subjects.

## Database Storage

The elementary subjects are pushed into the `elementary_subjects_dev` database.

In [4]:
BRAND = 'columbuscafe_test'

BRAND_DESCR = '''
Feedbacks are from client of columbus cafe.
Columbus Café & Co est une chaîne française de cafés.
'''

In [5]:
# getting feedbacks from mongo

from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
        },
    },
])))

from_mongo.shape

(2994, 16)

In [6]:
# keeping only feedbacks with extractions and splitted_analysis_v2

subdf = from_mongo[from_mongo['extractions'].notna()]
subdf = subdf[subdf['splitted_analysis_v2'].notna()]
print(subdf.shape)

row = subdf.sample().iloc[0]

print(row.verbatim['text'])

(59, 16)
Service courtois. Préparation des beignets par un employé exposé comme un poisson dans un aquarium. Perso, je n'aimerais pas trop travaillé exposé comme ça. Enfin. Bref. Petit espace sympa pour manger sur place, bien situé, sous un puit de lumière. 😎 Produits  d'excellente qualité. 👍 En rapport avec 💵 dépensé. Ça fait une bonne pause goûter au cours des courses.


In [7]:
extractions = get_field_value(
    feedback_id=row['_id'],
    field_name='extractions',
)

extractions

[{'sentiment': 'POSITIVE',
  'extraction': 'Service courtois',
  'text': 'Service courtois.',
  'elementary_subjects': ["Service : Qualité de l'équipe"],
  'topics': []},
 {'sentiment': 'NEGATIVE',
  'extraction': 'Préparation des beignets exposée de manière peu agréable',
  'text': ' Préparation des beignets par un employé exposé comme un poisson dans un aquarium.',
  'elementary_subjects': ['Produits : Problème de présentation : Présentation des beignets peu agréable'],
  'topics': []},
 {'sentiment': 'NEGATIVE',
  'extraction': 'Préparation des beignets exposée de manière peu agréable',
  'text': " je n'aimerais pas trop travaillé exposé comme ça.",
  'elementary_subjects': ['Produits : Problème de présentation : Présentation des beignets peu agréable'],
  'topics': []},
 {'sentiment': 'POSITIVE',
  'extraction': 'Petit espace sympa pour manger sur place',
  'text': ' Petit espace sympa pour manger sur place,',
  'elementary_subjects': ['Ambiance : Ambiance agréable'],
  'topics': [

The function `classify_one_feedback` is used to classify each extraction.

In [10]:
res = classify_one_feedback(
    feedback_id=row['_id'],
    extractions=extractions,
    model="gpt-4o-mini",
    brand=BRAND,
    brand_descr=BRAND_DESCR,
    language="french",
    extractions_column="extractions",
    update_mongo=True
)

res

Updating feedback columbuscafe_test/d04ec381e414f002aaf3 with updates: {'extractions': [{'sentiment': 'POSITIVE', 'extraction': 'Personnel drôle et sympathique', 'text': ' et je suis tombée sur un personnel drôle et sympathique !', 'elementary_subjects': ["Service : Qualité de l'équipe"], 'topics': []}, {'sentiment': 'NEGATIVE', 'extraction': "Prix élevés pour les budgets d'étudiant", 'text': ' Les prix sont un peu élevés pour les budgets d’étudiant.', 'elementary_subjects': ['Prix : Tarifs élevés'], 'topics': [{'topic': {'1': 'Prix', '2': 'Prix des produits'}, 'classification_scheme_id': ObjectId('66df25bf365a29ed53caf5a2'), 'classification_scheme_name': 'Test Classification Scheme'}]}, {'sentiment': 'POSITIVE', 'extraction': 'Recommandation du matcha coco', 'text': ' mais je recommande le matcha coco (:', 'elementary_subjects': ['Produits : Recommandation de produits : Recommandation du matcha coco'], 'topics': []}], 'splitted_analysis_v2': [{'text': 'Je suis venue au Columbus Café e

{'id': 'columbuscafe_test/d04ec381e414f002aaf3',
 'extractions': [{'sentiment': 'POSITIVE',
   'extraction': 'Personnel drôle et sympathique',
   'text': ' et je suis tombée sur un personnel drôle et sympathique !',
   'elementary_subjects': ["Service : Qualité de l'équipe"],
   'topics': []},
  {'sentiment': 'NEGATIVE',
   'extraction': "Prix élevés pour les budgets d'étudiant",
   'text': ' Les prix sont un peu élevés pour les budgets d’étudiant.',
   'elementary_subjects': ['Prix : Tarifs élevés'],
   'topics': [{'topic': {'1': 'Prix', '2': 'Prix des produits'},
     'classification_scheme_id': ObjectId('66df25bf365a29ed53caf5a2'),
     'classification_scheme_name': 'Test Classification Scheme'}]},
  {'sentiment': 'POSITIVE',
   'extraction': 'Recommandation du matcha coco',
   'text': ' mais je recommande le matcha coco (:',
   'elementary_subjects': ['Produits : Recommandation de produits : Recommandation du matcha coco'],
   'topics': []}]}

In [11]:
# this function allows to parallelize the analysis process and to save the results on the mongo database
from utils.analysis_utils import run_analysis_full_parallel

In [12]:
to_analyse = subdf.sample(10) # test on a subset
print('To extract:', to_analyse.shape[0])

extractions_with_ids = to_analyse[['extractions', '_id']].to_dict(orient='records')

To extract: 10


In [13]:
analysis = run_analysis_full_parallel(
    extractions_with_ids,
    BRAND,
    brand_descr=BRAND_DESCR,
    language='french',
    model="gpt-4o-mini",
    save_to_mongo=True
)

analysis

Processing chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Updating feedback columbuscafe_test/1976c554cdd362249798 with updates: {'extractions': [], 'splitted_analysis_v2': [], 'topics_v2': []}
Updating feedback columbuscafe_test/5c9bb9d227a8028911e2 with updates: {'extractions': [{'sentiment': 'POSITIVE', 'extraction': 'Appréciation générale', 'text': '1👍'}], 'splitted_analysis_v2': [{'text': '1👍', 'extractions': [{'sentiment': 'POSITIVE', 'extraction': 'Appréciation générale', 'text': '1👍'}]}], 'topics_v2': []}
Updating feedback columbuscafe_test/5b6e404652649c8ae22e with updates: {'extractions': [{'sentiment': 'POSITIVE', 'extraction': 'Appréciation générale', 'text': '2👍', 'elementary_subjects': ['Satisfaction générale : Volonté de revenir'], 'topics': []}], 'splitted_analysis_v2': [{'text': '2👍', 'extractions': [{'sentiment': 'POSITIVE', 'extraction': 'Appréciation générale', 'text': '2👍', 'elementary_subjects': ['Satisfaction générale : Volonté de revenir'], 'topics': []}]}], 'topics_v2': []}
Updating feedback columbuscafe_test/ece98e66

Processing chunks: 100%|██████████| 1/1 [00:18<00:00, 18.97s/it]

[check_duplicates()] invalid syntax (<string>, line 4)


[{'id': 'columbuscafe_test/5b6e404652649c8ae22e',
  'extractions': [{'sentiment': 'POSITIVE',
    'extraction': 'Appréciation générale',
    'text': '2👍',
    'elementary_subjects': ['Satisfaction générale : Volonté de revenir'],
    'topics': []}]},
 {'id': 'columbuscafe_test/72f750f6fe9766ce0e77',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Erreur dans la commande',
    'text': 'Encore une fois une erreur dans la commande,',
    'elementary_subjects': ['Service : Problème de traitement des commandes : Commande non traitée'],
    'topics': []},
   {'sentiment': 'NEGATIVE',
    'extraction': 'Problèmes récurrents avec les commandes sur Deliveroo',
    'text': ' comme à chaque commande réalisée sur Deliveroo.',
    'elementary_subjects': ['Service : Problème de traitement des commandes : Problèmes avec les commandes sur Deliveroo'],
    'topics': []},
   {'sentiment': 'POSITIVE',
    'extraction': 'Service généralement bon en boutique',
    'text': ' Cliente de la bou

In [14]:
analysis

[{'id': 'columbuscafe_test/5b6e404652649c8ae22e',
  'extractions': [{'sentiment': 'POSITIVE',
    'extraction': 'Appréciation générale',
    'text': '2👍',
    'elementary_subjects': ['Satisfaction générale : Volonté de revenir'],
    'topics': []}]},
 {'id': 'columbuscafe_test/72f750f6fe9766ce0e77',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Erreur dans la commande',
    'text': 'Encore une fois une erreur dans la commande,',
    'elementary_subjects': ['Service : Problème de traitement des commandes : Commande non traitée'],
    'topics': []},
   {'sentiment': 'NEGATIVE',
    'extraction': 'Problèmes récurrents avec les commandes sur Deliveroo',
    'text': ' comme à chaque commande réalisée sur Deliveroo.',
    'elementary_subjects': ['Service : Problème de traitement des commandes : Problèmes avec les commandes sur Deliveroo'],
    'topics': []},
   {'sentiment': 'POSITIVE',
    'extraction': 'Service généralement bon en boutique',
    'text': ' Cliente de la bou

## Check elementary subjects that we generated

In [1]:
from utils.database import get_elementary_subjects

In [16]:
positive_elementary_subjects = get_elementary_subjects(BRAND, 'positive')

len(positive_elementary_subjects)

21

In [17]:
[x['elementary_subject'] for x in positive_elementary_subjects[:5]]

['Ambiance : Ambiance agréable',
 'Service : Accueil chaleureux',
 'Service : Problemlösung : Schnelle Problemlösung bei Fehlern',
 'Produits : Qualité des produits : Qualité de la nourriture',
 'Produits : Qualité des produits : Qualité du café']

In [18]:
negative_elementary_subjects = get_elementary_subjects(BRAND, 'negative')

print(len(negative_elementary_subjects))

[x['elementary_subject'] for x in negative_elementary_subjects[:5]]

15


['Service : Comportement du personnel : Serveur impoli ou nerveux',
 'Magasin : Aménagement : Emplacement du mobilier inapproprié',
 "Accessibilité : Difficulté d'accès pour les fauteuils roulants",
 'Produits : Problème de qualité : Chocolat non consommable',
 'Prix : Tarifs élevés']